Notebook này trình bày các bước làm sạch và chuẩn bị dữ liệu giao dịch thẻ tín dụng để tiếp tục đào tạo GNN với GraphSAGE.

Mục tiêu:
 * Làm sạch dữ liệu
   * Convert tên các trường dữ liệu thành một từ duy nhất
     * tên trường không được sử dụng trong GNN, nó giúp truy cập các trường dễ dàng hơn trong quá trình làm sạch  
   * Encode các trường categorical
     * sử dụng one-hot encoding cho các trường có ít hơn 8 categories
     * sử dụng binary encoding cho các trường có nhiều hơn 8 categories
   * Tạo một chỉ mục nút liên tục trên người dùng, thương gia và giao dịch
     * việc có ID nút bắt đầu từ số không và sau đó liền kề là rất quan trọng để tạo dữ liệu theo định dạng Hàng thưa nén (Compressed Sparse Row - CSR) mà không lãng phí bộ nhớ
 * Produce:
   * Với XGBoost:
     * Training   - dữ liệu trước 2018
     * Validation - dữ liệu trong 2018
     * Test.      - dữ liệu sau 2018
   * Với GNN
     * Dữ liệu huấn luyện
       * Danh sách các cạnh
       * Dữ liệu đặc trưng
   * Test - dữ liệu sau 2018



Graph formation

Với việc chúng ta chỉ giới hạn ở dữ liệu trong tệp giao dịch, mô hình lý tưởng sẽ là có một đồ thị hai phía của Người dùng đến Nhà cung cấp, trong đó các cạnh biểu diễn giao dịch thẻ tín dụng và sau đó thực hiện Link Classification trên các cạnh để xác định gian lận. Thật không may, phiên bản cuGraph hiện tại không hỗ trợ GNN Link Prediction. May mắn thay, xem các giao dịch dưới dạng các nút và sau đó thực hiện node classification bằng GraphSAGE GNN phổ biến. Đồ thị được tạo ra sẽ là đồ thị ba phía, trong đó mỗi giao dịch được biểu diễn dưới dạng một nút.

Các đặc trưng:

Đối với phương pháp XGBoost, không cần tạo các đặc trưng trống cho Merchants. Tuy nhiên, đối với xử lý GNN, mọi nút cần có cùng một tập dữ liệu đặc trưng. Do đó, chúng ta cần tạo các đặc trưng trống cho các nút User và Merchant.

# Tải dữ liệu

In [1]:
import os
import gdown

file_id = '16IziiOifwF5n9ZkqjYp7E34NxnE-NcwR' 
url = f'https://drive.google.com/uc?id={file_id}'
output = 'card-transaction.zip'

if not os.path.exists(output):
    print("Downloading ZIP dataset...")
    gdown.download(url, output, quiet=False)
else:
    print(f"ZIP file already exists at '{output}', skipping download.")

Downloading...
From (original): https://drive.google.com/uc?id=16IziiOifwF5n9ZkqjYp7E34NxnE-NcwR
From (redirected): https://drive.google.com/uc?id=16IziiOifwF5n9ZkqjYp7E34NxnE-NcwR&confirm=t&uuid=15929a63-691b-4374-848d-fa4835472c12
To: /home/hkk1907/VNPT/fraud_detection/fraud_detection/ai_credit_fraud_workflow/card-transaction.zip
100%|██████████| 276M/276M [00:27<00:00, 9.94MB/s] 


In [2]:
import zipfile
zip_file = 'card-transaction.zip'
extract_folder = 'data/TabFormer/raw/'
os.makedirs(extract_folder, exist_ok=True)
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Cài đặt các thư viện


Ta dùng cuDF và thực hiện hầu hết tiền xử lý dữ liệu trên GPU

In [3]:
!pip install category-encoders==2.6.4 -q


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import json
import os
import pickle

import cudf
import numpy as np
import pandas as pd
import scipy.stats as ss
from category_encoders import BinaryEncoder
from scipy.stats import pointbiserialr
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler

/home/hkk1907/miniconda3/envs/ai_credit_fraud_workflow/lib/python3.11/site-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/home/hkk1907/miniconda3/envs/ai_credit_fraud_workflow/lib/python3.11/site-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)


CUDARuntimeError: cudaErrorInsufficientDriver: CUDA driver version is insufficient for CUDA runtime version

# Định nghĩa một số đối số

In [ ]:
# Đồ thị là vô hướng
make_undirected = True

# Có nên phân bổ các tính năng trên các nút Người dùng và Thương gia không?
spread_features = False

# Chỉ định có lấy mẫu không đầy đủ (undersampling) dữ liệu lớp đa số (non-fraud) hay không.
under_sample = True

# Tỷ lệ giữa số giao dịch gian lận và tổng số giao dịch trong tập dữ liệu sau khi under-sampling.
fraud_ratio = 0.1

In [ ]:
tabformer_base_path = './data/TabFormer'
tabformer_raw_file_path = os.path.join(tabformer_base_path, 'raw', 'card_transaction.v1.csv')
tabformer_xgb = os.path.join(tabformer_base_path, 'xgb')
tabformer_gnn = os.path.join(tabformer_base_path, 'gnn')

if not os.path.exists(tabformer_xgb):
    os.makedirs(tabformer_xgb)
if not os.path.exists(tabformer_gnn):
    os.makedirs(tabformer_gnn)

# Tải và hiểu dữ liệu

In [ ]:
data = cudf.read_csv(tabformer_raw_file_path)

In [ ]:
data.head(5)

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,<NA>,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,<NA>,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,<NA>,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,<NA>,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,<NA>,No


In [ ]:
data.columns

Index(['User', 'Card', 'Year', 'Month', 'Day', 'Time', 'Amount', 'Use Chip',
       'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
       'Errors?', 'Is Fraud?'],
      dtype='object')

* Các trường categorical có thứ tự - 'Year', 'Month', 'Day'
* Các trường categorical không có thứ tự - 'User', 'Card', 'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC', 'Errors?'
* Nhãn - 'Is Fraud?'

In [ ]:
data.isnull().sum()

User                     0
Card                     0
Year                     0
Month                    0
Day                      0
Time                     0
Amount                   0
Use Chip                 0
Merchant Name            0
Merchant City            0
Merchant State     2720821
Zip                2878135
MCC                      0
Errors?           23998469
Is Fraud?                0
dtype: int64

In [ ]:
100*data.isnull().sum()/len(data)

User               0.000000
Card               0.000000
Year               0.000000
Month              0.000000
Day                0.000000
Time               0.000000
Amount             0.000000
Use Chip           0.000000
Merchant Name      0.000000
Merchant City      0.000000
Merchant State    11.156896
Zip               11.801972
MCC                0.000000
Errors?           98.407215
Is Fraud?          0.000000
dtype: float64

* Đối với nhiều giao dịch, 'Merchant State' và 'Zip' bị thiếu, nhưng thật tốt khi tất cả các giao dịch đều có 'Merchant City' được chỉ định.
* Hơn 98% các giao dịch bị thiếu dữ liệu cho các trường 'Errors?'.

# Lưu một vài giao dịch trước khi thực hiện biến đổi (để cho inference)

In [ ]:
out_path = os.path.join(tabformer_xgb, 'example_transactions.csv')
data.tail(10).to_pandas().to_csv(out_path, header=True, index=False)

# Đổi tên các cột thành 1 từ duy nhất và sử dụng biến cho tên cột để truy cập dễ dàng hơn

In [ ]:
COL_USER = 'User'
COL_CARD = 'Card'
COL_AMOUNT = 'Amount'
COL_MCC = 'MCC'
COL_TIME = 'Time'
COL_DAY = 'Day'
COL_MONTH = 'Month'
COL_YEAR = 'Year'

COL_MERCHANT = 'Merchant'
COL_STATE ='State'
COL_CITY ='City'
COL_ZIP = 'Zip'
COL_ERROR = 'Errors'
COL_CHIP = 'Chip'
COL_FRAUD = 'Fraud'

In [ ]:
_ = data.rename(columns={
    "Merchant Name": COL_MERCHANT,
    "Merchant State": COL_STATE,
    "Merchant City": COL_CITY,
    "Errors?": COL_ERROR,
    "Use Chip": COL_CHIP,
    "Is Fraud?": COL_FRAUD
    },
    inplace=True
)

In [ ]:
data.head(5)

,User,Card,Year,Month,Day,Time,Amount,Chip,Merchant,City,State,Zip,MCC,Errors,Fraud
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,<NA>,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,<NA>,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,<NA>,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,<NA>,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,<NA>,No


# Xử lý missing values
* Zip là numeral, thay thế missing bởi 0
* State và Error là string, thay thế missing bởi 'XX'

In [ ]:
UNKNOWN_STRING_MARKER = 'XX'
UNKNOWN_ZIP_CODE = 0

In [ ]:
# Đảm bảo rằng 'XX' không tồn tại trong trường State và Error trước khi thay thế các giá trị bị thiếu bởi 'XX'
assert(UNKNOWN_STRING_MARKER not in set(data[COL_STATE].unique().to_pandas()))
assert(UNKNOWN_STRING_MARKER not in set(data[COL_ERROR].unique().to_pandas()))

In [ ]:
# Đảm bảo rằng 0 or 0.0 không tồn tại trong trường Zip trước khi thay thế các giá trị bị thiếu bởi 0
assert(float(0) not in set(data[COL_ZIP].unique().to_pandas()))
assert(0 not in set(data[COL_ZIP].unique().to_pandas()))

In [ ]:
data[COL_STATE] = data[COL_STATE].fillna(UNKNOWN_STRING_MARKER)
data[COL_ERROR] = data[COL_ERROR].fillna(UNKNOWN_STRING_MARKER)
data[COL_ZIP] = data[COL_ZIP].fillna(UNKNOWN_ZIP_CODE)

In [ ]:
data.isnull().sum()

User        0
Card        0
Year        0
Month       0
Day         0
Time        0
Amount      0
Chip        0
Merchant    0
City        0
State       0
Zip         0
MCC         0
Errors      0
Fraud       0
dtype: int64

# Làm sạch trường Amount field
* Xóa "$" trong trường Amount và conver từ string thành float
* Phân tích sự phân bố của trường Amount và chọn  scaler phù hợp

In [ ]:
data[COL_AMOUNT] = data[COL_AMOUNT].str.replace("$","").astype("float")

In [ ]:
data[COL_AMOUNT].describe()

count    2.438690e+07
mean     4.363401e+01
std      8.202239e+01
min     -5.000000e+02
25%      9.200000e+00
50%      3.014000e+01
75%      6.506000e+01
max      1.239050e+04
Name: Amount, dtype: float64

# Xem sự khác nhau giữa giao dịch fraud và non-fraud

In [ ]:
# Fraud
data[COL_AMOUNT][data[COL_FRAUD]=='Yes'].describe()

count    29757.000000
mean       108.590874
std        201.167421
min       -500.000000
25%         18.360000
50%         71.020000
75%        150.130000
max       5694.440000
Name: Amount, dtype: float64

In [ ]:
# Non-fraud
data[COL_AMOUNT][data[COL_FRAUD]=='No'].describe()

count    2.435714e+07
mean     4.355465e+01
std      8.173917e+01
min     -5.000000e+02
25%      9.200000e+00
50%      3.011000e+01
75%      6.500000e+01
max      1.239050e+04
Name: Amount, dtype: float64

## Findings
* 25th percentile = 9.2
* 75th percentile =  65
* Median là khoảng 30 và mean khoảng 43 trong khi giá trị max value là hơn 1200 và min là -500
* Số tiền trung bình trong các giao dịch gian lận > 2x số tiền trung bình trong các giao dịch không gian lận

Ta cần scale dữ liệu, và RobustScaler có thể là lựa chọn tốt.

# Trường "Fraud"

In [ ]:
# Có bao nhiêu danh mục khác nhau trong cột COL_FRAUD?
# Hy vọng là chỉ có hai danh mục, 'Yes' và 'No'
data[COL_FRAUD].unique()

0     No
1    Yes
Name: Fraud, dtype: object

In [ ]:
data[COL_FRAUD].value_counts()

Fraud
No     24357143
Yes       29757
Name: count, dtype: int64

In [ ]:
100 * data[COL_FRAUD].value_counts()/len(data)

Fraud
No     99.87798
Yes     0.12202
Name: count, dtype: float64

## Đổi giá trị 'Fraud' thành integer
  * 1 == Fraud
  * 0 == Non-fraud

In [ ]:
fraud_to_binary = {'No': 0, 'Yes': 1}
data[COL_FRAUD] = data[COL_FRAUD].map(fraud_to_binary).astype('int8')

In [ ]:
data[COL_FRAUD].value_counts()

Fraud
0    24357143
1       29757
Name: count, dtype: int64

# Cột 'City', 'State', and 'Zip'

In [ ]:
data[COL_CITY].unique()

0               La Verne
1          Monterey Park
2                 ONLINE
3              Mira Loma
4            Diamond Bar
              ...       
13424          Loysville
13425    Laurel Bloomery
13426            Alburgh
13427            Buskirk
13428             Mooers
Name: City, Length: 13429, dtype: object

In [ ]:
data[COL_STATE].unique()

0                            CA
1                            XX
2                            NE
3                            IL
4                            MO
                 ...           
219    Central African Republic
220                       Qatar
221    East Timor (Timor-Leste)
222                  Seychelles
223                     Andorra
Name: State, Length: 224, dtype: object

In [ ]:
data[COL_ZIP].unique()

0        91750.0
1        91754.0
2        91755.0
3            0.0
4        91752.0
          ...   
27317    17047.0
27318    37680.0
27319     5440.0
27320    12028.0
27321    12958.0
Name: Zip, Length: 27322, dtype: float64

# Cột 'Chip'





In [ ]:
data[COL_CHIP].unique()

0     Swipe Transaction
1    Online Transaction
2      Chip Transaction
Name: Chip, dtype: object

# Cột 'Error'

In [ ]:
data[COL_ERROR].unique()

0                                                    XX
1                                     Technical Glitch,
2                                 Insufficient Balance,
3                                              Bad PIN,
4                         Bad PIN,Insufficient Balance,
5                                       Bad Expiration,
6                             Bad PIN,Technical Glitch,
7                                      Bad Card Number,
8                                              Bad CVV,
9                                          Bad Zipcode,
10               Insufficient Balance,Technical Glitch,
11                Bad Card Number,Insufficient Balance,
12                             Bad Card Number,Bad CVV,
13                        Bad CVV,Insufficient Balance,
14                      Bad Card Number,Bad Expiration,
15                              Bad Expiration,Bad CVV,
16                 Bad Expiration,Insufficient Balance,
17                     Bad Expiration,Technical 

In [ ]:
# Xóa ','
data[COL_ERROR] = data[COL_ERROR].str.replace(",","")

#### Findings
Chúng ta có thể  one hot hoặc binary encode với ít categories và binary/hash encode với nhiều hơn 8 categories

# Time
Thời gian được ghi lại theo giờ:phút.

Converting thời gian thành số phút.

time = (số giờ * 60) + số phút

In [ ]:
data[COL_TIME].describe()

count     24386900
unique        1440
top          12:31
freq         30604
Name: Time, dtype: object

In [ ]:
# Chia cột time thành hours và minutes ép sang int32
T = data[COL_TIME].str.split(':', expand=True)
T[0] = T[0].astype('int32')
T[1] = T[1].astype('int32')

In [ ]:
# Thay thế cột 'Time' bởi cột mới
data[COL_TIME] = (T[0] * 60 ) + T[1]
data[COL_TIME] = data[COL_TIME].astype("int32")

In [ ]:
data.head(5)

,User,Card,Year,Month,Day,Time,Amount,Chip,Merchant,City,State,Zip,MCC,Errors,Fraud
0,0,0,2002,9,1,381,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,XX,0
1,0,0,2002,9,1,402,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,XX,0
2,0,0,2002,9,2,382,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,XX,0
3,0,0,2002,9,2,1065,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,XX,0
4,0,0,2002,9,3,383,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,XX,0


In [ ]:
# Xóa DataFrame tạm thời
del(T)


# Cột Merchant

In [ ]:
data[COL_MERCHANT]

0           3527213246127876953
1           -727612092139916043
2           -727612092139916043
3           3414527459579106770
4           5817218446178736267
                   ...         
24386895   -5162038175624867091
24386896   -5162038175624867091
24386897    2500998799892805156
24386898    2500998799892805156
24386899    4751695835751691036
Name: Merchant, Length: 24386900, dtype: int64

## Convert kiểu thành string

In [ ]:
data[COL_MERCHANT] = data[COL_MERCHANT].astype('str')

# Hơn 100000 thương gia
data[COL_MERCHANT].unique()

0          3527213246127876953
1          -727612092139916043
2          3414527459579106770
3          5817218446178736267
4         -7146670748125200898
                  ...         
100338     2963633013590132543
100339     3970346884766028008
100340    -4348891722741102135
100341     -642409450154660123
100342    -3533580464561517260
Name: Merchant, Length: 100343, dtype: object

# Cột Card
* "Card 0" của User 1 khác "Card 0" của User 2.
* Kết hợp User và Card để (User, Card) là duy nhất

In [ ]:
data[COL_CARD].unique()

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
Name: Card, dtype: int64

In [ ]:
max_nr_cards_per_user = len(data[COL_CARD].unique())

In [ ]:
max_nr_cards_per_user

9

In [ ]:
# Kết hợp User vaf Card để tạo ra số duy nhất
data[COL_CARD] = data[COL_USER] * len(data[COL_CARD].unique())  + data[COL_CARD]
data[COL_CARD] = data[COL_CARD].astype('int')

In [ ]:
data[COL_CARD].unique()

0           0
1           1
2           2
3           3
4           4
        ...  
6134    17974
6135    17975
6136    17982
6137    17991
6138    17992
Name: Card, Length: 6139, dtype: int64

In [ ]:
data[COL_USER].unique()

0          0
1          1
2          2
3          3
4          4
        ... 
1995    1995
1996    1996
1997    1997
1998    1998
1999    1999
Name: User, Length: 2000, dtype: int64

# Định nghĩa hàm để tính tương quan giữa các trường categorical khác nhau với target

In [ ]:
# https://en.wikipedia.org/wiki/Cram%C3%A9r's_V

def cramers_v(x, y):
    confusion_matrix = cudf.crosstab(x, y).to_numpy()
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(k-1, r-1))))

## Tính tương quan của các trường với target

In [ ]:
data

,User,Card,Year,Month,Day,Time,Amount,Chip,Merchant,City,State,Zip,MCC,Errors,Fraud
0,0,0,2002,9,1,381,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,XX,0
1,0,0,2002,9,1,402,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,XX,0
2,0,0,2002,9,2,382,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,XX,0
3,0,0,2002,9,2,1065,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,XX,0
4,0,0,2002,9,3,383,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,XX,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,17992,2020,2,27,1343,-54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,XX,0
24386896,1999,17992,2020,2,27,1344,54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,XX,0
24386897,1999,17992,2020,2,28,463,59.15,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,XX,0
24386898,1999,17992,2020,2,28,1210,43.12,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,XX,0


In [ ]:
sparse_factor = 1
columns_to_compute_corr =  [COL_CARD, COL_CHIP, COL_ERROR, COL_STATE, COL_CITY, COL_ZIP, COL_MCC, COL_MERCHANT, COL_USER, COL_DAY, COL_MONTH, COL_YEAR]
for c1 in columns_to_compute_corr:
    for c2 in [COL_FRAUD]:
        coff =  100 * cramers_v(data[c1][::sparse_factor], data[c2][::sparse_factor])
        print('Correlation ({}, {}) = {:6.2f}%'.format(c1, c2, coff))

Correlation (Card, Fraud) =   6.59%
Correlation (Chip, Fraud) =   5.63%
Correlation (Errors, Fraud) =   1.81%
Correlation (State, Fraud) =  35.92%
Correlation (City, Fraud) =  32.47%
Correlation (Zip, Fraud) =  14.99%
Correlation (MCC, Fraud) =  12.70%
Correlation (Merchant, Fraud) =  34.88%
Correlation (User, Fraud) =   3.40%
Correlation (Day, Fraud) =   0.26%
Correlation (Month, Fraud) =   0.23%
Correlation (Year, Fraud) =   2.35%


# Tương quan của target với các cột numerical

In [ ]:
# https://en.wikipedia.org/wiki/Point-biserial_correlation_coefficient
# Sử dụng hệ số tương quan Point-biserial (rpb) để kiểm tra xem
# các cột số có quan trọng để dự đoán xem một giao dịch có phải là gian lận hay không
for col in [COL_TIME, COL_AMOUNT]:
    r_pb, p_value = pointbiserialr(data[COL_FRAUD].to_pandas(), data[col].to_pandas())
    print('r_pb ({}) = {:3.2f} with p_value {:3.2f}'.format(col, r_pb, p_value))

r_pb (Time) = -0.00 with p_value 0.00
r_pb (Amount) = 0.03 with p_value 0.00


## Findings
* Time không phải yếu tố dự đoán quan trọng
* Amount có 3% tương quan với target

# Dựa vào correlation, chọn 1 tập các trường để dự đoán giao dịch có là gian lận hay không

In [ ]:
# Vì mối tương quan chéo giữa Fraud với Day, Month, Year thấp hơn đáng kể,
# chúng ta có thể bỏ qua chúng ngay bây giờ và thêm các tính năng này sau.
numerical_predictors = [COL_AMOUNT]
nominal_predictors = [COL_ERROR, COL_CARD, COL_CHIP, COL_CITY, COL_ZIP, COL_MCC, COL_MERCHANT]

predictor_columns = numerical_predictors + nominal_predictors

target_column = [COL_FRAUD]

# Xóa các điểm dữ liệu non-fraud trùng lặp

In [ ]:
# Xoá các điểm dữ liệu trùng lặp
fraud_data = data[data[COL_FRAUD] == 1]
data = data[data[COL_FRAUD] == 0]
data = data.drop_duplicates(subset=nominal_predictors)
data = cudf.concat([data, fraud_data])

In [ ]:
100*data[COL_FRAUD].value_counts()/len(data)

Fraud
0    98.378669
1     1.621331
Name: count, dtype: float64

# Chia dữ liệu
 * Training   - tất cả dữ liệu trước 2018
 * Validation - tất cả dữ liệu trong 2018
 * Test.      - tất cả dữ liệu sau 2018

In [ ]:
if under_sample:
    fraud_df = data[data[COL_FRAUD]==1]
    non_fraud_df = data[data[COL_FRAUD]==0]
    nr_non_fraud_samples = min((len(data) - len(fraud_df)), int(len(fraud_df)/fraud_ratio))
    data = cudf.concat([fraud_df, non_fraud_df.sample(nr_non_fraud_samples)])

training_idx = data[COL_YEAR] < 2018
validation_idx = data[COL_YEAR] == 2018
test_idx = data[COL_YEAR] > 2018

data[COL_FRAUD].value_counts()

Fraud
0    297570
1     29757
Name: count, dtype: int64

# Scale các cột numerical và encode các cột categorical của training data

In [ ]:
# Vì một số bộ mã hóa mà chúng ta muốn sử dụng không có sẵn trong cuml
# nên chúng ta có thể sử dụng pandas ngay bây giờ.
# Chuyển training data thành pandas để tiền xử lý
pdf_training = data[training_idx].to_pandas()[predictor_columns + target_column]

In [ ]:
# Sử dụng one-hot encoding cho các cột <= 8 categories, và binary encoding cho các cột nhiều categories hơn
columns_for_binary_encoding = []
columns_for_onehot_encoding = []
for col in nominal_predictors:
    print(col, len(data[col].unique()))
    if len(data[col].unique()) <= 8:
        columns_for_onehot_encoding.append(col)
    else:
        columns_for_binary_encoding.append(col)

Errors 22
Card 6060
Chip 3
City 11355
Zip 22778
MCC 109
Merchant 45218


In [ ]:
# Đánh dấu cột categorical là "category"
pdf_training[nominal_predictors] = pdf_training[nominal_predictors].astype("category")

In [ ]:
# encoders để encode các cột categorical và scalers để scale các cột numerical

bin_encoder = Pipeline(
    steps=[
        ("binary", BinaryEncoder(handle_missing='value', handle_unknown='value'))
    ]
)
onehot_encoder = Pipeline(
    steps=[
        ("onehot", OneHotEncoder())
    ]
)
std_scaler = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("standard", StandardScaler())],
)
robust_scaler = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("robust", RobustScaler())],
)

In [ ]:
# Kết hợp encoders và scalers trong cột transformer
transformer = ColumnTransformer(
    transformers=[
        ("binary", bin_encoder, columns_for_binary_encoding),
        ("onehot", onehot_encoder, columns_for_onehot_encoding),
        ("robust", robust_scaler, [COL_AMOUNT]),
    ], remainder="passthrough"
)

In [ ]:
# Fit cột transformer với training data

pd.set_option('future.no_silent_downcasting', True)
transformer = transformer.fit(pdf_training[predictor_columns])

In [ ]:
# biến đổi các tên cột
columns_of_transformed_data = list(
    map(lambda name: name.split('__')[1],
        list(transformer.get_feature_names_out(predictor_columns))))

In [ ]:
columns_of_transformed_data

['Errors_0',
 'Errors_1',
 'Errors_2',
 'Errors_3',
 'Errors_4',
 'Card_0',
 'Card_1',
 'Card_2',
 'Card_3',
 'Card_4',
 'Card_5',
 'Card_6',
 'Card_7',
 'Card_8',
 'Card_9',
 'Card_10',
 'Card_11',
 'Card_12',
 'City_0',
 'City_1',
 'City_2',
 'City_3',
 'City_4',
 'City_5',
 'City_6',
 'City_7',
 'City_8',
 'City_9',
 'City_10',
 'City_11',
 'City_12',
 'City_13',
 'Zip_0',
 'Zip_1',
 'Zip_2',
 'Zip_3',
 'Zip_4',
 'Zip_5',
 'Zip_6',
 'Zip_7',
 'Zip_8',
 'Zip_9',
 'Zip_10',
 'Zip_11',
 'Zip_12',
 'Zip_13',
 'Zip_14',
 'MCC_0',
 'MCC_1',
 'MCC_2',
 'MCC_3',
 'MCC_4',
 'MCC_5',
 'MCC_6',
 'Merchant_0',
 'Merchant_1',
 'Merchant_2',
 'Merchant_3',
 'Merchant_4',
 'Merchant_5',
 'Merchant_6',
 'Merchant_7',
 'Merchant_8',
 'Merchant_9',
 'Merchant_10',
 'Merchant_11',
 'Merchant_12',
 'Merchant_13',
 'Merchant_14',
 'Merchant_15',
 'Chip_Chip Transaction',
 'Chip_Online Transaction',
 'Chip_Swipe Transaction',
 'Amount']

In [ ]:
# kiểu dữ liệu của các cột được biến đổi
type_mapping = {}
for col in columns_of_transformed_data:
    if col.split('_')[0] in nominal_predictors:
        type_mapping[col] = 'int8'
    elif col in numerical_predictors:
        type_mapping[col] = 'float'
    elif col in target_column:
        type_mapping[col] = data.dtypes.to_dict()[col]

In [ ]:
# biến đổi training data
preprocessed_training_data = transformer.transform(pdf_training[predictor_columns])

# Convert dữ liệu được biến đổi thành panda DataFrame
preprocessed_training_data = pd.DataFrame(
    preprocessed_training_data, columns=columns_of_transformed_data)
# Copy cột target
preprocessed_training_data[COL_FRAUD] = pdf_training[COL_FRAUD].values
preprocessed_training_data = preprocessed_training_data.astype(type_mapping)

In [ ]:
# Lưu lại transformer

with open(os.path.join(tabformer_base_path, 'preprocessor.pkl'),'wb') as f:
    pickle.dump(transformer, f)

# Lưu training data đã được biến đổi cho huấn luyện XGBoost

In [ ]:
with open(os.path.join(tabformer_base_path, 'preprocessor.pkl'),'rb') as f:
    loaded_transformer = pickle.load(f)

In [ ]:
# Biến đổi test data sử dụng transformer fitted trên training data
pdf_test = data[test_idx].to_pandas()[predictor_columns + target_column]
pdf_test[nominal_predictors] = pdf_test[nominal_predictors].astype("category")

preprocessed_test_data = loaded_transformer.transform(pdf_test[predictor_columns])
preprocessed_test_data = pd.DataFrame(preprocessed_test_data, columns=columns_of_transformed_data)

# Copy cột target
preprocessed_test_data[COL_FRAUD] = pdf_test[COL_FRAUD].values
preprocessed_test_data = preprocessed_test_data.astype(type_mapping)

In [ ]:
# Biến đổi validation data sử dụng transformer fitted trên training data
pdf_validation = data[validation_idx].to_pandas()[predictor_columns + target_column]
pdf_validation[nominal_predictors] = pdf_validation[nominal_predictors].astype("category")

preprocessed_validation_data = loaded_transformer.transform(pdf_validation[predictor_columns])
preprocessed_validation_data = pd.DataFrame(preprocessed_validation_data, columns=columns_of_transformed_data)

# Copy cột target
preprocessed_validation_data[COL_FRAUD] = pdf_validation[COL_FRAUD].values
preprocessed_validation_data = preprocessed_validation_data.astype(type_mapping)

# Viết ra dữ liệu cho XGB

In [ ]:
# Training data
out_path = os.path.join(tabformer_xgb, 'training.csv')
if not os.path.exists(os.path.dirname(out_path)):
  os.makedirs(os.path.dirname(out_path))
preprocessed_training_data.to_csv(out_path, header=True, index=False, columns=columns_of_transformed_data + target_column)
# preprocessed_training_data.to_parquet(out_path, index=False, compression='gzip')

In [ ]:
# validation data
out_path = os.path.join(tabformer_xgb, 'validation.csv')
if not os.path.exists(os.path.dirname(out_path)):
  os.makedirs(os.path.dirname(out_path))
preprocessed_validation_data.to_csv(out_path, header=True, index=False, columns=columns_of_transformed_data + target_column)
# preprocessed_validation_data.to_parquet(out_path, index=False, compression='gzip')

In [ ]:
# test data
out_path = os.path.join(tabformer_xgb, 'test.csv')
preprocessed_test_data.to_csv(out_path, header=True, index=False, columns=columns_of_transformed_data + target_column)
# preprocessed_test_data.to_parquet(out_path, index=False, compression='gzip')

In [ ]:
# Viết dữ liệu thử nghiệm chưa chuyển đổi chỉ có các cột dự đoán (được đổi tên) và mục tiêu
out_path = os.path.join(tabformer_xgb, 'untransformed_test.csv')
pdf_test.to_csv(out_path, header=True, index=False)

In [ ]:
# Xóa dataFrames không cần thiết nữa
del(pdf_training)
del(pdf_validation)
del(pdf_test)
del(preprocessed_training_data)
del(preprocessed_validation_data)
del(preprocessed_test_data)

# Dữ liệu GNN

## Cài đặt các ID đỉnh
Để tạo 1 graph, các đỉnh khác nhau cần được chỉ định ID đỉnh duy nhất. Hơn nữa, các ID cần liên tiếp nhau và phải dương.

Có 3 nhóm node: Transactions, Users, và Merchants.

Các ID này không được sử dụng trong training, chỉ sử dụng để xử lý graph.

In [ ]:
# Sử dụng training data giống với sử dụng cho XGBoost
data = data[training_idx]

In [ ]:
# rất nhiều quá trình đã xảy ra, sắp xếp dữ liệu và thiết lập lại chỉ mục
data = data.sort_values(by=[COL_YEAR, COL_MONTH, COL_DAY, COL_TIME], ascending=False)
data.reset_index(inplace=True, drop=True)

In [ ]:
# Mỗi transaction có ID duy nhất
COL_TRANSACTION_ID = 'Tx_ID'
COL_MERCHANT_ID = 'Merchant_ID'
COL_USER_ID = 'User_ID'

# Số transaction giống với size của list, và là giá trị index
data[COL_TRANSACTION_ID] = data.index

In [ ]:
# Lấy transaction ID lớn nhất để tính merchant ID đầu tiên
max_tx_id = data[COL_TRANSACTION_ID].max()

In [ ]:
# Convert chuỗi Merchant thành các số nguyên liên tiếp
merchant_name_to_id = dict((v, k) for k, v in data[COL_MERCHANT].unique().to_dict().items())
data[COL_MERCHANT_ID] = data[COL_MERCHANT].map(merchant_name_to_id) + (max_tx_id + 1)
data[COL_MERCHANT_ID].min(), data[COL_MERCHANT].max()

(np.int64(280988), '999657473293735')

In [ ]:
# Lấy merchant ID lớn nhất để tính user ID đầu tiên
max_merchant_id = data[COL_MERCHANT_ID].max()

NOTE: Các cột 'User' và 'Card' của dữ liệu ban đầu được sử dụng để tạo cột 'Card' đã cập nhật. Bạn có thể dùng user hoặc card như các node.

In [ ]:
# Convert Card thành các ID liên tiếp
id_to_consecutive_id = dict((v, k) for k, v in data[COL_CARD].unique().to_dict().items())
data[COL_USER_ID] = data[COL_CARD].map(id_to_consecutive_id) + max_merchant_id + 1
data[COL_USER_ID].min(), data[COL_USER_ID].max()

# id_to_consecutive_id = dict((v, k) for k, v in data[COL_USER].unique().to_dict().items())
# data[COL_USER_ID] = data[COL_USER].map(id_to_consecutive_id) + max_merchant_id + 1
# data[COL_USER_ID].min(), data[COL_USER].max()

(np.int64(322103), np.int64(326877))

In [ ]:
# Lưu user ID lớn nhất
max_user_id = data[COL_USER_ID].max()

In [ ]:
# Kiểm tra transaction, merchant và user ids là liên tiếp
id_range = data[COL_TRANSACTION_ID].min(), data[COL_TRANSACTION_ID].max()
print(f'Transaction ID range {id_range}')
id_range = data[COL_MERCHANT_ID].min(), data[COL_MERCHANT_ID].max()
print(f'Merchant ID range {id_range}')
id_range = data[COL_USER_ID].min(), data[COL_USER_ID].max()
print(f'User ID range {id_range}')

Transaction ID range (np.int64(0), np.int64(280987))
Merchant ID range (np.int64(280988), np.int64(322102))
User ID range (np.int64(322103), np.int64(326877))


In [ ]:
assert( data[COL_TRANSACTION_ID].max() == data[COL_MERCHANT_ID].min() - 1)
assert( data[COL_MERCHANT_ID].max() == data[COL_USER_ID].min() - 1)
assert(len(data[COL_USER_ID].unique()) == (data[COL_USER_ID].max() - data[COL_USER_ID].min() + 1))
assert(len(data[COL_MERCHANT_ID].unique()) == (data[COL_MERCHANT_ID].max() - data[COL_MERCHANT_ID].min() + 1))
assert(len(data[COL_TRANSACTION_ID].unique()) == (data[COL_TRANSACTION_ID].max() - data[COL_TRANSACTION_ID].min() + 1))

## Viết ra dữ liệu cho GNN

### Create the Graph Edge Data file
The file is in COO format

In [ ]:
COL_GRAPH_SRC = 'src'
COL_GRAPH_DST = 'dst'
COL_GRAPH_WEIGHT = 'wgt'

# User tới Transactions
U_2_T = cudf.DataFrame()
U_2_T[COL_GRAPH_SRC] = data[COL_USER_ID]
U_2_T[COL_GRAPH_DST] = data[COL_TRANSACTION_ID]
if make_undirected:
  T_2_U = cudf.DataFrame()
  T_2_U[COL_GRAPH_SRC] = data[COL_TRANSACTION_ID]
  T_2_U[COL_GRAPH_DST] = data[COL_USER_ID]
  U_2_T = cudf.concat([U_2_T, T_2_U])
  del T_2_U

In [ ]:
# Transactions tới Merchants
T_2_M = cudf.DataFrame()
T_2_M[COL_GRAPH_SRC] = data[COL_MERCHANT_ID]
T_2_M[COL_GRAPH_DST] = data[COL_TRANSACTION_ID]

if make_undirected:
  M_2_T = cudf.DataFrame()
  M_2_T[COL_GRAPH_SRC] = data[COL_TRANSACTION_ID]
  M_2_T[COL_GRAPH_DST] = data[COL_MERCHANT_ID]
  T_2_M = cudf.concat([T_2_M, M_2_T])
  del M_2_T

In [ ]:
Edge = cudf.concat([U_2_T, T_2_M])
Edge[COL_GRAPH_WEIGHT] = 0.0
len(Edge)

1123952

In [ ]:
# viết dữ liệu
out_path = os.path.join (tabformer_gnn, 'edges.csv')

if not os.path.exists(os.path.dirname(out_path)):
  os.makedirs(os.path.dirname(out_path))

Edge.to_csv(out_path, header=False, index=False)

In [ ]:
del(Edge)
del(U_2_T)
del(T_2_M)

## Dữ liệu đặc trưng
Dữ liệu đặc trưng cần được sắp xếp theo thứ tự, trong đó chỉ số hàng tương ứng với ID nút

Dữ liệu bao gồm 3 tập đặc trưng:
* Transactions
* Users
* Merchants

### Để có các vector đặc trưng của các nút Transaction, biến đổi training data sử dụng pre-fitted transformer

In [ ]:
node_feature_df = pd.DataFrame(
    loaded_transformer.transform(
        data[predictor_columns].to_pandas()
        ),
    columns=columns_of_transformed_data).astype(type_mapping)

node_feature_df[COL_FRAUD] = data[COL_FRAUD].to_pandas()

### Đối với các nút liên kết với merchant và user, hãy thêm các vectơ đặc trưng bằng không

In [ ]:
# Số lượng nút cho users và merchants
nr_users_and_merchant_nodes = max_user_id - max_tx_id

In [ ]:
if not spread_features:
    # Tạo vector đặc trưng 0 cho mỗi nút user và merchant
    empty_feature_df = cudf.DataFrame(
        columns=columns_of_transformed_data + target_column,
        dtype='int8',
        index=range(nr_users_and_merchant_nodes)
    )
    empty_feature_df = empty_feature_df.fillna(0)
    empty_feature_df=empty_feature_df.astype(type_mapping)

In [ ]:
if not spread_features:
    # Concatenate các đặc trưng transaction theo sau là các đặc trưng dành cho nút merchants và user
    node_feature_df = pd.concat([node_feature_df, empty_feature_df.to_pandas()]).astype(type_mapping)

In [ ]:
# Các cột cụ thể của User
if spread_features:
    user_specific_columns = [COL_CARD, COL_CHIP]
    user_specific_columns_of_transformed_data = []

    for col in node_feature_df.columns:
        if col.split('_')[0] in user_specific_columns:
            user_specific_columns_of_transformed_data.append(col)

In [ ]:
# Các cột cụ thể của Merchant
if spread_features:
    merchant_specific_columns = [COL_MERCHANT, COL_CITY, COL_ZIP, COL_MCC]
    merchant_specific_columns_of_transformed_data = []

    for col in node_feature_df.columns:
        if col.split('_')[0] in merchant_specific_columns:
            merchant_specific_columns_of_transformed_data.append(col)

In [ ]:
# Các cột cụ thể của Transaction
if spread_features:
    transaction_specific_columns = list(
        set(numerical_predictors).union(nominal_predictors)
        - set(user_specific_columns).union(merchant_specific_columns))
    transaction_specific_columns_of_transformed_data = []

    for col in node_feature_df.columns:
        if col.split('_')[0] in transaction_specific_columns:
            transaction_specific_columns_of_transformed_data.append(col)

### Xây dựng vector đặc trưng cho merchants

In [ ]:
if spread_features:
    # Tìm chỉ số của các thương gia duy nhất
    idx_df = cudf.DataFrame()
    idx_df[COL_MERCHANT_ID] =  data[COL_MERCHANT_ID]
    idx_df = idx_df.sort_values(by=COL_MERCHANT_ID)
    idx_df = idx_df.drop_duplicates(subset=COL_MERCHANT_ID)
    assert((data.iloc[idx_df.index][COL_MERCHANT_ID] == idx_df[COL_MERCHANT_ID]).all())

In [ ]:
if spread_features:
    # Copy các cột cụ thể của merchant và cho phần còn lại là 0
    merchant_specific_feature_df = node_feature_df.iloc[idx_df.index.to_numpy()]
    merchant_specific_feature_df.\
    loc[:,
        transaction_specific_columns_of_transformed_data +
          user_specific_columns_of_transformed_data] = 0.0


In [ ]:
if spread_features:
    # Tìm chỉ số của các users duy nhất
    idx_df = cudf.DataFrame()
    idx_df[COL_USER_ID] = data[COL_USER_ID]
    idx_df = idx_df.sort_values(by=COL_USER_ID)
    idx_df = idx_df.drop_duplicates(subset=COL_USER_ID)
    assert((data.iloc[idx_df.index][COL_USER_ID] == idx_df[COL_USER_ID]).all())

In [ ]:
if spread_features:
    # Copy các cột cụ thể của user và cho phần còn lại là 0
    user_specific_feature_df = node_feature_df.iloc[idx_df.index.to_numpy()]
    user_specific_feature_df.\
    loc[:,
        transaction_specific_columns_of_transformed_data +
          merchant_specific_columns_of_transformed_data] = 0.0

In [ ]:
# Concatenate các đặc trưng của nút user và merchant
if spread_features:

    node_feature_df[merchant_specific_columns_of_transformed_data] = 0.0
    node_feature_df[user_specific_columns_of_transformed_data] = 0.0
    node_feature_df = pd.concat(
        [node_feature_df, merchant_specific_feature_df, user_specific_feature_df]
        ).astype(type_mapping)

    # lưu các đặc trưng
    node_feature_df = node_feature_df[
        transaction_specific_columns_of_transformed_data +
        merchant_specific_columns_of_transformed_data +
        user_specific_columns_of_transformed_data + [COL_FRAUD]]

In [ ]:
# lưu label
label_df = node_feature_df[[COL_FRAUD]]

In [ ]:
# Xóa label khỏi vectỏr đặc trưng
_ = node_feature_df.drop(columns=[COL_FRAUD], inplace=True)

### Viết đặc trưng nút và label

In [ ]:
# Viết nút label vào file csv
out_path = os.path.join(tabformer_gnn, 'labels.csv')

if not os.path.exists(os.path.dirname(out_path)):
  os.makedirs(os.path.dirname(out_path))

label_df.to_csv(out_path, header=False, index=False)
# label_df.to_parquet(out_path, index=False, compression='gzip')

In [ ]:
# Viết nút đặc trưng vào file csv
out_path = os.path.join(tabformer_gnn, 'features.csv')

if not os.path.exists(os.path.dirname(out_path)):
  os.makedirs(os.path.dirname(out_path))
node_feature_df[columns_of_transformed_data].to_csv(out_path, header=True, index=False)
# node_feature_df.to_parquet(out_path, index=False, compression='gzip')

In [ ]:
# Xóa dataFrames
del data
del node_feature_df
del label_df

if spread_features:
    del merchant_specific_feature_df
    del user_specific_feature_df
else:
    del empty_feature_df

## Số các nút transaction trong training data

In [ ]:
# Số các nút transaction cần cho training GNN
nr_transaction_nodes = max_tx_id + 1
nr_transaction_nodes

np.int64(280988)

### Số lượng card tối đa cho mỗi user

In [ ]:
# Số lượng card tối đa cho mỗi user, cần cho inference
max_nr_cards_per_user

9

### Lưu biến cho training and inference

In [ ]:
variables_to_save = {
    k: v for k, v in globals().items() if isinstance(v, (str, int)) and k.startswith('COL_')}

In [ ]:
variables_to_save['NUM_TRANSACTION_NODES'] = int(nr_transaction_nodes)
variables_to_save['MAX_NR_CARDS_PER_USER'] = int(max_nr_cards_per_user)

In [ ]:
# Lưu dictionary thành file JSON

with open(os.path.join(tabformer_base_path, 'variables.json'), 'w') as json_file:
    json.dump(variables_to_save, json_file, indent=4)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/data /content/drive/MyDrive/VNPT/Fraud_Detection